In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

np.random.seed(42)
n = 5000

df = pd.DataFrame({
    '지점': np.random.choice(['서울', '부산', '대전'], n),
    '장소': np.random.choice(['실내', '실외'], n),
    '상태': np.random.choice(['정상', '주의', '위험'], n),
    '장비명': np.random.choice(['A1', 'B2', 'C3'], n),
    '온도': np.random.normal(25, 5, n),
    '진동': np.random.normal(0.5, 0.2, n),
    '전류': np.random.normal(3.0, 0.5, n),
    '압력': np.random.normal(1.0, 0.1, n),
    '시작시간': pd.date_range('2024-01-01', periods=n, freq='H')
})
df['종료시간'] = df['시작시간'] + pd.Timedelta(hours=1)
df['에너지사용량'] = df['온도']*0.5 + df['전류']*1.2 + np.random.normal(0, 0.5, n)

C:\Users\3\AppData\Local\Temp\ipykernel_106092\2996910454.py:22: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  '시작시간': pd.date_range('2024-01-01', periods=n, freq='H')


In [2]:
df

,지점,장소,상태,장비명,온도,진동,전류,압력,시작시간,종료시간,에너지사용량
0,대전,실내,주의,B2,24.118187,0.383603,2.439801,1.086833,2024-01-01 00:00:00,2024-01-01 01:00:00,15.853302
1,서울,실내,위험,A1,17.365554,0.411542,2.698062,0.784365,2024-01-01 01:00:00,2024-01-01 02:00:00,11.701820
2,대전,실내,위험,A1,23.684701,0.682598,3.510641,0.965160,2024-01-01 02:00:00,2024-01-01 03:00:00,16.111446
3,대전,실외,정상,C3,24.218521,0.447242,3.115501,1.085519,2024-01-01 03:00:00,2024-01-01 04:00:00,15.356875
4,서울,실외,위험,A1,16.508494,0.518519,3.180125,0.813375,2024-01-01 04:00:00,2024-01-01 05:00:00,12.118880
...,...,...,...,...,...,...,...,...,...,...,...
4995,서울,실내,주의,A1,24.905006,0.543409,3.604103,0.741683,2024-07-27 03:00:00,2024-07-27 04:00:00,16.374590
4996,부산,실외,위험,C3,24.252701,0.503841,2.802598,1.038421,2024-07-27 04:00:00,2024-07-27 05:00:00,15.041399
4997,대전,실내,정상,B2,19.819118,0.441876,3.850387,0.976770,2024-07-27 05:00:00,2024-07-27 06:00:00,15.273830
4998,부산,실내,주의,B2,27.725716,0.359282,3.382686,1.075058,2024-07-27 06:00:00,2024-07-27 07:00:00,18.514228


In [ ]:
#문자열 처리
for col in ['지점','장소','상태','장비명']:
    df[col] = LabelEncoder().fit_transform(df[col])
#피처선택
features = ['지점','장소','상태','장비명','온도','진동','전류','압력']
target = '에너지사용량'
#정규화
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
x_raw = df[features].values
y_raw = df[target].values
x_scaled = scaler_x.fit_transform(x_raw)
y_scaled = scaler_y.fit_transform(y_raw.reshape(-1, 1))
data_scaled = np.hstack([x_scaled,y_scaled])
#시퀀스
def create_seq(data,seq_len,max_n): 
    xs,ns,ys = [],[],[]
    #시계열에서 시퀀스를 생성할 때 슬라이딩 윈도우 방식 사용
    #len(data) 전체 시계열 데이터수 , seq_len 모델에 들어갈 과거 시간 길이 ex:10시간  max_n 예측할 최대 미래시간 
    for i in range(len(data)-seq_len-max_n):  # - seq_len - max_n의미 i가 10시간 만큼 가져오고 i:i+seq_len인데 뒤에 seq_len+max_n만큼 여유 주는것
        x = data[i:i+seq_len,:-1] #i부터 i+seq_len만큼만 데이터 즉 피쳐값만 가져오고 마지막 열은 :-1임으로 안가져옴
        for n in range(1,max_n+1):#가져온 x의 데이터를 여러 시간차 의 타켓y를 만듬
            y = data[i+seq_len+n-1,-1] #n시간뒤 target값
            xs.append(x)
            ns.append([n])
            ys.append(y)
    return np.array(xs),np.array(ns),np.array(ys)
#데이터셋구성
class TimeSeriesDataset(Dataset):
    def __init__(self,x,n_array,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.n = torch.tensor(n_array,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index],self.n[index],self.y[index]
#모델구성
class LSTMwithN(nn.Module):
    def __init__(self,input_dim,hidden_dim,num_layer):
        # input_dim 입력 피쳐 수  hidden_dim LSTM 내부 뉴런 수 num_layers LSTM 층 수
        # batch_first=True	입력 shape을 (batch, seq_len, input_dim)으로 설정
        # fc = nn.Linear(hidden_dim, 1)	마지막 hidden state를 1개의 출력값(에너지 예측)으로 변환
        super.__init__()
        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layer,batch_first=True)
        self.fc_n = nn.Linear(1,hidden_dim) 
        self.fc = nn.Linear(hidden_dim,1)

    def forward(self,x,n):
        # x: (batch, seq_len, input_dim)
        # n: (batch, 1)
        #out, (h_n, c_n) = self.lstm(x)
        #out =  모든 시점의 hidden state	(batch, seq_len, hidden_dim) 
        # hidden_dim = LSTM이 각 시점에서 기억하고 있는 정보(메모리)의 요약본
        #h_n	마지막 시점 hidden state (각 레이어별)	(num_layers, batch, hidden_dim)
        #c_n	마지막 시점 cell state (각 레이어별)	(num_layers, batch, hidden_dim)
        out,_ = self.lstm(x) #모든 시점의 LSTM 출력 → shape: (batch, seq_len, hidden_dim)
        last_out = out[:,-1,:] #out[:, -1, :] → 64개 시퀀스 중에서 각자의 10번째 시점의 hidden state 
        n_feat = self.fc_n(n)
        combined = torch.cat([last_out,n_feat],dim=1)
        return self.fc(combined).squeeze()
#학습구성
#예측구성
#평가

[[0.47881083]
 [0.24988331]
 [0.4930458 ]
 ...
 [0.44685666]
 [0.62554377]
 [0.589188  ]]
